In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.layers import LSTM, Activation, Dropout, Dense, Input, Embedding, Layer, Attention
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adadelta


import keras.backend as kb
from keras.backend import clear_session
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from tensorflow import keras

import sys
sys.path.append("../")

from pre_processing import creazione_modello_GloVe

In [2]:
dataset = pickle.load(open("dataset/dataset_SLS.pickle", "rb"))

In [3]:
Y = np.array(dataset["sentiment"])

In [4]:
x_train,x_test,y_train,y_test = train_test_split(dataset["sentence"],Y, test_size=0.1, shuffle=True)

In [5]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(2700,) (2700,)
(300,) (300,)


In [6]:
print("numero di positivi nel train: ", list(y_train).count(1))
print("numero di negativi nel train: ", list(y_train).count(0))

print("numero di positivi nel test: ", list(y_test).count(1))
print("numero di negativi nel test: ", list(y_test).count(0))

numero di positivi nel train:  1344
numero di negativi nel train:  1356
numero di positivi nel test:  156
numero di negativi nel test:  144


In [7]:
embedding = creazione_modello_GloVe("dataset/glove.6B.50d.txt")

In [8]:
tokenizer = Tokenizer(num_words=140000)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [9]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banana"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=300, weights=[embedding_matrix])

In [10]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=kb.squeeze(kb.tanh(kb.dot(x,self.W)+self.b),axis=-1)
        at=kb.softmax(et)
        at=kb.expand_dims(at,axis=-1)
        output=x*at
        return kb.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [11]:
lstmAtt_model = Sequential()
lstmAtt_model.add(embedding_layer)
lstmAtt_model.add(LSTM(32, return_sequences=True))
lstmAtt_model.add(attention())
lstmAtt_model.add(Dense(2, activation="softmax"))

lstmAtt_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
lstmAtt_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 50)           249600    
_________________________________________________________________
lstm (LSTM)                  (None, 300, 32)           10624     
_________________________________________________________________
attention (attention)        (None, 32)                332       
_________________________________________________________________
dense (Dense)                (None, 2)                 66        
Total params: 260,622
Trainable params: 260,622
Non-trainable params: 0
_________________________________________________________________


In [12]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=300)

In [13]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)

In [16]:
lstmAtt_model.fit(x_train_index[:2100], y_train_categorical[:2100], epochs=15, verbose=1, batch_size=192, validation_data=(x_train_index[2100:], y_train_categorical[2100:]))

Epoch 1/15
 116/2100 [>.............................] - ETA: 2:21 - loss: 0.6439 - accuracy: 0.6293

KeyboardInterrupt: 

In [16]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=300)

In [17]:
y_pred = lstmAtt_model.predict(x_test_index)

In [18]:
print(classification_report(y_test, np.argmax(y_pred, axis=1).astype("float32")))

              precision    recall  f1-score   support

           0       0.79      0.71      0.75       150
           1       0.74      0.81      0.77       150

    accuracy                           0.76       300
   macro avg       0.77      0.76      0.76       300
weighted avg       0.77      0.76      0.76       300



In [ ]:
lstmAtt_model.save("models/SLS_LSTMAtt")